In [1]:
from __future__ import annotations

from pathlib import Path
WORK_DIR = Path.cwd()

import numpy as np
import pandas as pd
import requests

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import smtplib

## Testing Alpha Vantage API

In [2]:
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key='UX2OT39HH6HK14LB')
data, meta_data = ts.get_intraday('AAPL')

In [3]:
import numpy as np
import pandas as pd
import requests
import time

class AlphaVantageDataLoader():
    def __init__(
                 self, 
                 key: str):
        self.key=key

    
    def load_stock_data(self,
        tickers: list = [],
        frequency: str = 'daily', 
        output_size: str = 'full'
        ) -> dict: 

        freq_dict = {'intraday': 'TIME_SERIES_INTRADAY',
                     'daily': 'TIME_SERIES_DAILY',
                     'weekly': 'TIME_SERIES_WEEKLY',
                     'monthly': 'TIME_SERIES_MONTHLY'}
        
        if type(tickers) == str:
            tickers = [tickers]

        start = time.time()
        meta_dict = {} 
        df_dict = {}
        count=0
        
        for num, ticker in tqdm(enumerate(tickers)):
            link = f'https://www.alphavantage.co/query?function={freq_dict[frequency]}&symbol={ticker}&outputsize={output_size}&apikey={self.key}'
            stock = requests.get(link)
            keys_list = list(stock.json().keys())
            
            try:
                data, metadata = stock.json()[keys_list[-1]], stock.json()[keys_list[0]]

                df = pd.DataFrame(data).T
                df.columns = [col.split(' ')[-1] for col in df.columns]
                df = df.apply(pd.to_numeric)

                meta_dict[ticker] = meta_data
                df_dict[ticker]= df
                count+=1
            except:
                pass

            #Alpha Vantage only allows 5 API call per minute
            if (count+1)%5==0: 
                end = time.time()
                wait_time = (start+60) - end
                time.sleep(wait_time)
                start = time.time()
        
        self.meta = meta_dict

        return df_dict


In [4]:
ticker = pd.read_csv(WORK_DIR/'TSX.txt', sep='\t')

In [5]:
import random

ticker_list = ticker.loc[(~ticker.Symbol.str.contains('PR'))&
                           (~ticker.Description.str.contains('ETF'))&
                           (~ticker.Description.str.contains('Fund'))&
                           (~ticker.Description.str.contains('hdg'))]['Symbol'].tolist()

sampled_ticker = random.sample(ticker_list, 10)

In [6]:
# Initially wanted to use intraday data but intraday from alpha vantage doesn't seem to work for canadian stocks
loader = AlphaVantageDataLoader(key='UX2OT39HH6HK14LB')
#stock_data = loader.load_stock_data(tickers=sampled_ticker)

In [7]:
sampled_ticker

['SBR.TO',
 'OTEX.TO',
 'TCT.UN.TO',
 'ENB.PF.G.TO',
 'ERE.UN.TO',
 'RS.TO',
 'CTX.TO',
 'BK.TO',
 'TECK.B.TO',
 'CWB.TO']

### Value Threshold Announcement`

In [78]:
higher_threshold = {'BB.TO': 20, 'CGX.TO': 16.5, 'BDLP.TO': 20}
lower_threshold = {'BB.TO': 10, 'CGX.TO': 15, 'BDLP.TO': 30}
report_ticker = ['BB.TO', 'CGX.TO', 'L.TO', 'BDLP.TO']

download_list = list(set(list(higher_threshold.keys()) + list(lower_threshold.keys()) + report_ticker))

In [9]:
# Initially wanted to use intraday data but intraday from alpha vantage doesn't seem to work for canadian stocks
loader = AlphaVantageDataLoader(key='UX2OT39HH6HK14LB')
stock_data = loader.load_stock_data(tickers=download_list)

4it [00:10,  2.64s/it]


In [79]:
stock_report = [] 
lower_threshold_report = [] 
higher_threshold_report = []

for ticker in stock_data.keys():
    df1 = stock_data[ticker].reset_index()
    df1['index'] = pd.to_datetime(df1['index'])
    df1.set_index('index', inplace=True)

    weekly_close = df1.resample('W')['close'].mean().tail(52)
    current_close = weekly_close.iloc[-1]
    
    arr = (weekly_close>weekly_close.shift(1)).astype(int).iloc[::-1].values

    if np.any(arr==0):
        consecutive_week_high = np.where(arr==0)[0][0]
    else:
        consecutive_week_high = np.where(arr==0)[0][0]
    
    delta = (stock_data[ticker].iloc[0]['close'] - stock_data[ticker].iloc[1]['close'])/stock_data[ticker].iloc[1]['close']*100
    close_price = stock_data[ticker].iloc[0]['close']
    ma50 = stock_data[ticker].iloc[0:50]['close'].mean()
    ma200 = stock_data[ticker].iloc[0:200]['close'].mean()
    str1 = f"{ticker}\nClose Price: {close_price}\n% Changes: {delta:.2f}%\nMA50: {ma50:.2f}\nMA200: {ma200:.2f}\nConsecutive Weeks High: {consecutive_week_high}"
    stock_report.append(str1)
    
################## Checking Threshold #################
for ticker, value in higher_threshold.items():
    if ticker in stock_data.keys():
        close_price = stock_data[ticker].iloc[0]['close']
        if close_price>value:
            str1 = f"{ticker}: Closing Price {close_price:.2f}CAD is above threshold of {value:.2f}CAD"
            higher_threshold_report.append(str1)

for ticker, value in lower_threshold.items():
    if ticker in stock_data.keys():
        close_price = stock_data[ticker].iloc[0]['close']
        if close_price<value:
            str1 = f"{ticker}: Closing Price {close_price:.2f}CAD is below threshold of {value:.2f}CAD"
            lower_threshold_report.append(str1)
            
missing_list = [ticker for ticker in download_list if ticker not in stock_data.keys()]

In [80]:
higher_threshold_report

[]

In [81]:
lower_threshold_report

[]

In [82]:
output_list = [] 

joined_report = "\n\n".join(stock_report)

output_list.append(joined_report)
if len(higher_threshold_report)>0:
    output_list.append("\n".join(higher_threshold_report))
    
if len(lower_threshold_report)>0:
    output_list.append("\n".join(lower_threshold_report))
    
output_list.append(f'{", ".join(missing_list)} are missing')
output = '\n\n#################################################################\n\n'.join(output_list)

In [84]:
print(output)

CGX.TO
Close Price: 16.39
% Changes: 1.05%
MA50: 13.64
MA200: 10.58
Consecutive Weeks High: 4

L.TO
Close Price: 76.0
% Changes: -0.13%
MA50: 71.77
MA200: 67.23
Consecutive Weeks High: 6

BB.TO
Close Price: 17.21
% Changes: 2.38%
MA50: 12.07
MA200: 10.65
Consecutive Weeks High: 4

#################################################################

BDLP.TO are missing


#### Sending Notification Email

In [ ]:
sender = "tuateststock@outlook.com" 
receiver = "wongsangaroon@gmail.com"
password = ("ImTestingStocks123") 
message = f'From: {sender}\nTo: {receiver}\nSubject: Stock Alert!!\n\nI Made It WORKKK!!\n'

In [ ]:

server = smtplib.SMTP('smtp-mail.outlook.com', 587)
server.starttls()
server.login(sender, password)
print("Login Success")
server.sendmail(sender, receiver, msg = message) 
print("Email was sent")
server.close()